In [ ]:
import gym
import gym_futbol
from stable_baselines3 import A2C
import matplotlib.pyplot as plt
import utils

# Basics

In [ ]:
env = gym.make("Futbol2v2-v1")
env.render()

In [ ]:
env.action_space

In [ ]:
from stable_baselines3.common.env_checker import check_env
check_env(env)

# Learning model

In [ ]:
import stable_baselines3.common.results_plotter as results_plotter
from stable_baselines3.common.env_util import make_vec_env

log_dir = "logs/"
timesteps = 100000

env = gym.make("Futbol2v2-v1")
env = make_vec_env(lambda: env, n_envs=4, monitor_dir=log_dir)
model = A2C("MlpPolicy", env, device='cpu')
model.learn(timesteps, progress_bar=True)
model.save("models/footbol2v2-a2c")

# Display model statistic

In [ ]:
results_plotter.plot_results([log_dir], timesteps, results_plotter.X_EPISODES, "Futbol2v2-v1")
display.display(plt.gcf())
display.clear_output(wait=True)

# Load model from file

In [ ]:
model = A2C.load("models/footbol2v2-a2c", device="cpu")

# Run models

## Pygame
Run game with model in new windows with pygame

In [ ]:
RES = WIDTH, HEIGHT = 600, 600
FPS = 60

env = gym.make("Futbol2v2-v1", width=WIDTH-2, height=HEIGHT-2, player_radius=15, ball_radius=5)
visualizer = utils.PygameVisualizer(env, FPS)
visualizer.run(model)

env.close()
visualizer.close()

## Matplotlib
Run model in notebook with matplotlib

In [ ]:
env = gym.make("Futbol2v2-v1")

visualizer = utils.MatplotlibVisualizer(env)
visualizer.run(model)

env.close()